In [86]:
import numpy as np
import pandas as pd

In [87]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

-------

In [88]:
outdir = './data/kinsler-2020-preprocessed-data/'

-------

# Preprocess Bulk Fitness Assay Data

Reformat and filter original data set to meet the expectations of our pipeline.

In [89]:
rawDataDir = './data/kinsler-2020-elife-data/'

#### Load & organize raw data

##### Load the raw Kinsler et al. (2023) barcode counts data table

(rows: variants; cols: barcode counts for each experimental treatment/rep/timept and variant metadata)

In [90]:
rawCountsTable = pd.read_csv(rawDataDir+'BarcodeCounts_merged+flaskswapcorrected_092519_withBCinfo.csv')
rawCountsTable

,1.4%-R1-T1,1.4%-R1-T2,1.4%-R1-T3,1.4%-R1-T4,1.4%-R2-T1,1.4%-R2-T2,1.4%-R2-T3,1.4%-R2-T4,1.4%-T0,1.5%-R1-T1,1.5%-R1-T2,1.5%-R1-T3,1.5%-R1-T4,1.5%-R2-T1,1.5%-R2-T2,1.5%-R2-T3,1.5%-R2-T4,1.5%-T0,1.6%-R1-T1,1.6%-R1-T2,1.6%-R1-T3,1.6%-R1-T4,1.6%-R2-T1,1.6%-R2-T2,1.6%-R2-T3,1.6%-R2-T4,1.6%-T0,1.7%-R1-T1,1.7%-R1-T2,1.7%-R1-T3,1.7%-R1-T4,1.7%-R2-T1,1.7%-R2-T2,1.7%-R2-T3,1.7%-R2-T4,1.7%-T0,1.8%-R1-T1,1.8%-R1-T2,1.8%-R1-T3,1.8%-R1-T4,1.8%-R2-T1,1.8%-R2-T2,1.8%-R2-T3,1.8%-R2-T4,1.8%-T0,13-0,13-0_pool,13-1-1,13-1-2,13-1-3,13-2-1,13-2-2,13-2-3,13-3-1,13-3-2,13-3-3,13-4-1,13-4-2,13-4-3,18-1-1,18-1-2,18-1-3,18-2-1,18-2-2,18-2-3,18-3-1,18-3-2,18-3-3,18-4-1,18-4-2,18-4-3,2.5%-R1-T1,2.5%-R1-T2,2.5%-R1-T3,2.5%-R1-T4,2.5%-R2-T1,2.5%-R2-T2,2.5%-R2-T3,2.5%-R2-T4,2.5%-T0,20-1-1,20-1-2,20-1-3,20-2-1,20-2-2,20-2-3,20-3-1,20-3-2,20-3-3,20-4-1,20-4-2,20-4-3,21+wt,21-0-1,21-0-2,21-0-4,21-1-1,21-1-2,21-1-3,21-2-1,21-2-2,21-2-3,21-3-1,21-3-2,21-3-3,21-4-1,21-4-2,21-4-3,23-0,23-1-1,23-1-2,23-1-3,23-2-1,23-2-2,23-2-3,23-3-1,23-3-2,23-3-3,23-4-1,23-4-2,23-4-3,3-0,3-1-1,3-1-2,3-1-3,3-2-1,3-2-2,3-2-3,3-3-1,3-3-2,3-3-3,3-4-1,3-4-2,3-4-3,6-0,6-1-1,6-1-2,6-1-3,6-2-1,6-2-2,6-2-3,6-3-1,6-3-2,6-3-3,6-4-1,6-4-2,6-4-3,A0,A1,A2,A3,A4,AA0,AA1,AA2,AA3,AA4,B0,B1,B2,B3,B4,BB0,BB1,BB2,BB3,BB4,Ben0.4-R1-T1,Ben0.4-R1-T2,Ben0.4-R1-T3,Ben0.4-R1-T4,Ben0.4-R2-T1,Ben0.4-R2-T2,Ben0.4-R2-T3,Ben0.4-R2-T4,Ben0.4-T0,Ben10-R1-T1,Ben10-R1-T2,Ben10-R1-T3,Ben10-R1-T4,Ben10-R2-T1,Ben10-R2-T2,Ben10-R2-T3,Ben10-R2-T4,Ben10-T0,Ben2-R1-T1,Ben2-R1-T2,Ben2-R1-T3,Ben2-R1-T4,Ben2-R2-T1,Ben2-R2-T2,Ben2-R2-T3,Ben2-R2-T4,Ben2-T0,C0,C1,C2,C3,C4,CC0,CC1,CC2,CC3,CC4,D0,D1,D2,D3,D4,DD0,DD1,DD2,DD3,DD4,E0,E1,E2,E3,E4,EE0,EE1,EE2,EE3,EE4,F0,F1,F2,F3,F4,FF0,FF1,FF2,FF3,FF4,G0,G1,G2,G3,G4,GG0,GG1,GG2,GG3,GG4,H0,H1,H2,H3,H4,I0,I1,I2,I3,I4,J0,J1,J2,J3,J4,K0,K1,K2,K3,K4,L0,L1,L2,L3,L4,M0,M1,M2,M3,M4,N0,N1,N2,N3,N4,O0,O1,O2,O3,O4,P0,P1,P2,P3,P4,Q0,Q1,Q2,Q3,Q4,R0,R1,R2,R3,R4,S0,S1,S2,S3,S4,T0,T1,T2,T3,T4,Tneg,U0,U1,U2,U3,U4,V0,V1,V2,V3,V4,W0,W1,W2,W3,W4,X0,X1,X2,X3,X4,Y0,Y1,Y2,Y3,Y4,Z0,Z1,Z2,Z3,Z4,additional_muts,barcode,class,gene,ploidy,type
0,174.0,109.0,85.0,68.0,3.0,91.0,56.0,42.0,22.0,74.0,85.0,64.0,70.0,66.0,69.0,66.0,68.0,51.0,121.0,113.0,120.0,245.0,102.0,212.0,146.0,116.0,56.0,130.0,227.0,122.0,109.0,108.0,141.0,85.0,142.0,95.0,102.0,181.0,184.0,216.0,176.0,239.0,45.0,42.0,69.0,2.0,10.0,512.0,700.0,594.0,594.0,403.0,104.0,319.0,299.0,361.0,51.0,70.0,59.0,90.0,133.0,122.0,40.0,65.0,481.0,365.0,102.0,574.0,486.0,516.0,380.0,27.0,19.0,17.0,24.0,10.0,11.0,7.0,6.0,20.0,773.0,2336.0,2192.0,2785.0,2493.0,2199.0,1855.0,1569.0,1775.0,954.0,1495.0,1990.0,0.0,3276.0,3105.0,3372.0,962.0,760.0,1889.0,1883.0,574.0,1691.0,985.0,1144.0,1945.0,1025.0,1070.0,1302.0,2786.0,2019.0,2406.0,1856.0,2104.0,2892.0,2817.0,1529.0,2292.0,1772.0,1172.0,1750.0,1686.0,111.0,244.0,376.0,223.0,183.0,164.0,177.0,81.0,95.0,104.0,101.0,69.0,79.0,212.0,190.0,140.0,1.0,65.0,1.0,38.0,40.0,50.0,51.0,23.0,24.0,22.0,5430.0,6191.0,8361.0,574.0,2148.0,5659.0,2713.0,3435.0,198.0,236.0,18106.0,2775.0,17050.0,10057.0,18295.0,5765.0,3076.0,1751.0,167.0,193.0,25.0,21.0,29.0,42.0,34.0,22.0,32.0,45.0,15.0,52.0,18.0,46.0,10.0,0.0,36.0,36.0,46.0,0.0,80.0,74.0,0.0,84.0,47.0,37.0,80.0,104.0,63.0,2367.0,8268.0,15562.0,1846.0,11209.0,5624.0,1263.0,739.0,272.0,312.0,2722.0,5916.0,3468.0,3720.0,2945.0,6186.0,1103.0,520.0,662.0,395.0,5251.0,11213.0,3158.0,908.0,4044.0,2456.0,7453.0,3962.0,1371.0,237.0,3315.0,8696.0,4901.0,7853.0,4079.0,4798.0,12477.0,21019.0,3361.0,1481.0,1065.0,5289.0,1933.0,249.0,495.0,3758.0,1879.0,442.0,299.0,607.0,1239.0,4131.0,1553.0,150.0,235.0,6639.0,6157.0,2544.0,380.0,466.0,1539.0,4670.0,2662.0,363.0,341.0,1013.0,9165.0,5227.0,1091.0,1569.0,9030.0,4746.0,0.0,870.0,1449.0,3110.0,10152.0,10870.0,1247.0,907.0,5903.0,11731.0,0.0,0.0,0.0,8704.0,6361.0,7231.0,780.0,1934.0,7391.0,11230.0,6963.0,1183.0,1828.0,4329.0,3765.0,4583.0,3246.0,314.0,2835.0,6727.0,3805.0,502.0,87.0,1371.0,4424.0,1987.0,1161.0,204.0,1859.0,3879.0,6424.0,1474.0,3.0,714.0,

#### Separate the dataset into variant and ancestor data

In [91]:
ancCountsTable = rawCountsTable[rawCountsTable['class'] == 'Ancestor']

In [92]:
mutCountsTable = rawCountsTable[rawCountsTable['class'] != 'Ancestor']

#### Extract the counts data (excluding metadata) from the variant and ancestor datasets

In [93]:
metadata_columns = ['additional_muts', 'barcode', 'class', 'gene', 'ploidy', 'type']

#### Extract the metadata for variants from the variants dataset

In [94]:
rawCounts = mutCountsTable.drop(columns=metadata_columns)

#### Drop certain exception samples

In [95]:
drop_samples = ['13-0_pool', 'Tneg', '21+wt', '21-0-2', '21-0-4'] # '21-0-1'

In [96]:
variantsInfo = mutCountsTable[metadata_columns].copy()  # this .copy() prevents variantsInfo from being seen as a slice internally
variantsInfo

,additional_muts,barcode,class,gene,ploidy,type
0,TIP1-upstream_point_variant; YKR012C-upstream_...,53,Diploid,Diploid,Diploid,Diploid
1,SEH1-missense_variant; ZIP1-missense_variant; ...,151,PKA,IRA1,Haploid,stop_gained
2,NotSequenced,262,NotSequenced,NotSequenced,NotSequenced,NotSequenced
3,NaN,273,PKA,IRA1,Haploid,frameshift_variant
4,NotSequenced,323,NotSequenced,NotSequenced,NotSequenced,NotSequenced
...,...,...,...,...,...,...
544,NaN,9000106,PKA,IRA1,Haploid,stop_gained
545,NaN,9000107,PKA,IRA1,Haploid,stop_gained
546,NaN,9000108,PKA,IRA1,Haploid,stop_gained
547,NaN,9000109,PKA,IRA1,Haploid,stop_gained


In [97]:
rawCounts = rawCounts.drop(columns=drop_samples)

#### Parse treatment, rep, and timepoint information from sample labels (column names)

In [98]:
sample_columns_raw = rawCounts.columns
sample_columns_raw.values

array(['1.4%-R1-T1', '1.4%-R1-T2', '1.4%-R1-T3', '1.4%-R1-T4',
       '1.4%-R2-T1', '1.4%-R2-T2', '1.4%-R2-T3', '1.4%-R2-T4', '1.4%-T0',
       '1.5%-R1-T1', '1.5%-R1-T2', '1.5%-R1-T3', '1.5%-R1-T4',
       '1.5%-R2-T1', '1.5%-R2-T2', '1.5%-R2-T3', '1.5%-R2-T4', '1.5%-T0',
       '1.6%-R1-T1', '1.6%-R1-T2', '1.6%-R1-T3', '1.6%-R1-T4',
       '1.6%-R2-T1', '1.6%-R2-T2', '1.6%-R2-T3', '1.6%-R2-T4', '1.6%-T0',
       '1.7%-R1-T1', '1.7%-R1-T2', '1.7%-R1-T3', '1.7%-R1-T4',
       '1.7%-R2-T1', '1.7%-R2-T2', '1.7%-R2-T3', '1.7%-R2-T4', '1.7%-T0',
       '1.8%-R1-T1', '1.8%-R1-T2', '1.8%-R1-T3', '1.8%-R1-T4',
       '1.8%-R2-T1', '1.8%-R2-T2', '1.8%-R2-T3', '1.8%-R2-T4', '1.8%-T0',
       '13-0', '13-1-1', '13-1-2', '13-1-3', '13-2-1', '13-2-2', '13-2-3',
       '13-3-1', '13-3-2', '13-3-3', '13-4-1', '13-4-2', '13-4-3',
       '18-1-1', '18-1-2', '18-1-3', '18-2-1', '18-2-2', '18-2-3',
       '18-3-1', '18-3-2', '18-3-3', '18-4-1', '18-4-2', '18-4-3',
       '2.5%-R1-T1', '2.5%-R1-T2', '2.5

In [99]:
# Load table that maps alpha-character labeled samples to their corresponding treatments and replicate numbers
sampleAlphaLabelInfo = pd.read_csv(rawDataDir+'sample-alphalabel_treatment-rep.csv')

In [100]:
samples_info = []
for sample in sample_columns_raw:
    sample_components = sample.split('-')
    #------------------------------
    if(len(sample_components) == 3):
        # Sample label is in {treatment}-R{rep#}-T{timept#} -or- {treatment}-{timept#}-{rep#} format
        treatment   = sample_components[0]
        rep         = int(sample_components[1][-1]) if 'R' in sample_components[1] else int(sample_components[2])
        timept      = int(sample_components[2][-1]) if 'T' in sample_components[2] else int(sample_components[1])
    elif(len(sample_components) == 2):
        # Sample label is in {treatment}-{timept0} format
        treatment   = sample_components[0]
        rep         = 0
        timept      = int(sample_components[1][-1])
    else:
        # Sample label is in {treatment/rep label}{timept0} format
        # Replicate num needs to be interpreted from treatment/rep label, 
        #   and label should be replaced with descriptive treatment label.
        label       = sample_components[0][:-1]
        treatment   = sampleAlphaLabelInfo[sampleAlphaLabelInfo['sample_label'] == label]['treatment'].values[0]
        rep         = int(sampleAlphaLabelInfo[sampleAlphaLabelInfo['sample_label'] == label]['rep'].values[0])
        timept      = int(sample_components[0][-1])
    #------------------------------
    sample_info = {'kinsler_sample_label': sample, 'treatment': treatment, 'rep': rep, 'time': timept}
    #------------------------------
    # Treatments with simply numeric labels are 'evolutionary conditions' (EC) treatments
    if(sample_info['treatment'].isdigit()):
        sample_info['treatment'] = 'EC'+sample_info['treatment']
    # Treatments that read simply 1.6% should be edited to 1.6%Gluc
    if(sample_info['treatment'] == '1.6%'):
        sample_info['treatment'] = '1.6%Gluc'
    #------------------------------    
    samples_info.append(sample_info)
#------------------------------
samplesInfo = pd.DataFrame(samples_info)

#### Reconcile handling of zero timepoints across treatments/replicates

For most treatments, there is a single zero timepoint that is interpreted as shared by all replicates of that treatment (at this point, the common zero timepoint is listed as replicate 0).
However, some treatments have a separate zero timepoint for each replicate, and other treatments don't have a zero timepoint listed.
This step reconciles the different handling of zero timepoints across treatments by doing the following:
- If a treatment does not have a zero timepoint -- i.e., the first timepoint is T=1 for each replicate -- then shift the timepoint labels down such that the first timepoint is labeled timepoint T=0.
- If a treatment has a single, shared zero timepoint, expand the table such that there is a separate copy of that zero timepoint data for each replicate.

This results in all treatments having a zero timepoint listing for each replicate.

In [101]:
for treatment in samplesInfo['treatment'].unique():
    #------------------------------
    # If the treatment data is missing the zero timepoint, this will relabel the first timepoint as timepoint zero. 
    treatmentInfo = samplesInfo[samplesInfo['treatment'] == treatment]
    treatment_timepts = treatmentInfo['time'].values
    treatment_timepts_relabeled = treatment_timepts - treatment_timepts.min()  
    samplesInfo.loc[samplesInfo['treatment'] == treatment, 'time'] = treatment_timepts_relabeled
    #------------------------------
    treatmentInfo = samplesInfo[samplesInfo['treatment'] == treatment] # reload in case timept vals shifted above
    if(treatmentInfo['time'].value_counts()[0] == 1):
        #------------------------------
        # There is a single zero timepoint that is the same for all replicates; expand the table.
        treatment_numReps = int(treatmentInfo['rep'].max())
        treatmentInfo_timept0 = treatmentInfo[treatmentInfo['time'] == 0]
        treatmentInfo_timept0_expanded = pd.concat([treatmentInfo_timept0] * treatment_numReps)
        treatmentInfo_timept0_expanded.loc[:, 'rep'] = range(1, treatment_numReps+1)
        # Drop the existing shared zero timept row for this treatment
        samplesInfo.drop(samplesInfo[ (samplesInfo['treatment'] == treatment) & (samplesInfo['time'] == 0) ].index, inplace=True)
        # Append the dataframe with the expanded replicate zero timept rows
        samplesInfo = pd.concat([samplesInfo, treatmentInfo_timept0_expanded])


In [102]:
# Store a new standardized label for each sample
samplesInfo['sample'] = samplesInfo['treatment'] +'-R'+ samplesInfo['rep'].astype(str) +'-T'+ samplesInfo['time'].astype(str)

In [103]:
# Store a new standardized label for each assay (unique serial transfer)
samplesInfo['assay'] = samplesInfo['treatment'] +'-R'+ samplesInfo['rep'].astype(str)

In [104]:
# Sort dataframe for convenience:
samplesInfo.sort_values(by=['treatment', 'rep', 'time'], ascending=[False, True, True], ignore_index=True, inplace=True)

#### Update the counts dataframes to reflect the changes from reconciliation of zero timepoints and standardized sample labels

In [105]:
rawCounts_updated = pd.DataFrame(index=rawCounts.index)

In [106]:
for new_sample_label in samplesInfo['sample'].values:
    kinsler_sample_label = samplesInfo[samplesInfo['sample'] == new_sample_label]['kinsler_sample_label'].values[0]
    rawCounts_updated[new_sample_label] = rawCounts[kinsler_sample_label]
rawCounts = rawCounts_updated.copy() # building the new df col-by-col results in a fragmented df; the .copy() here resolves that

#### Reorder columns for cosmetic reasons

In [107]:
samplesInfo = samplesInfo[['kinsler_sample_label', 'sample', 'assay', 'treatment', 'rep', 'time']]

In [108]:
samplesInfo

,kinsler_sample_label,sample,assay,treatment,rep,time
0,A0,M3-R1-T0,M3-R1,M3,1,0
1,A1,M3-R1-T1,M3-R1,M3,1,1
2,A2,M3-R1-T2,M3-R1,M3,1,2
3,A3,M3-R1-T3,M3-R1,M3,1,3
4,A4,M3-R1-T4,M3-R1,M3,1,4
...,...,...,...,...,...,...
349,O0,0.2MKCl-R1-T0,0.2MKCl-R1,0.2MKCl,1,0
350,O1,0.2MKCl-R1-T1,0.2MKCl-R1,0.2MKCl,1,1
351,O2,0.2MKCl-R1-T2,0.2MKCl-R1,0.2MKCl,1,2
352,O3,0.2MKCl-R1-T3,0.2MKCl-R1,0.2MKCl,1,3


In [109]:
samplesInfo['assay'].nunique()

72

In [110]:
samplesInfo['treatment'].nunique()

33

----------
#### Filter samples by total counts

In [111]:
# Create a column in samplesInfo to indicate which samples are excluded from analysis.
samplesInfo['exclude_from_analysis'] = False

In [112]:
MIN_SAMPLE_COUNT_TOTAL = 1e5

In [113]:
for sample in samplesInfo['sample']:
    sample_countTotal = rawCounts[sample].sum()
    samplesInfo.loc[samplesInfo['sample'] == sample, 'total_raw_count'] = sample_countTotal
# display(samplesInfo)

In [114]:
# Exclude treatments that have one or more samples with total count below threshold 
good_coverage_samples = []
for treatment in samplesInfo['treatment'].unique():
    treatmentInfo = samplesInfo[samplesInfo['treatment'] == treatment]
    if(np.all((treatmentInfo['time'] == 0) | (treatmentInfo['total_raw_count'] >= MIN_SAMPLE_COUNT_TOTAL))):
        good_coverage_samples.extend( treatmentInfo['sample'].values )
    else:
        samplesInfo.loc[samplesInfo['treatment'] == treatment, 'exclude_from_analysis'] = True

In [115]:
rawCounts    = rawCounts[good_coverage_samples]

#### Manually exclude other abnormal assays/treatments

In [116]:
drop_treatments = ['1.7%', '1%Gly', '2ugFlu', '8.5uMGdA']
for drop_treatment in drop_treatments:
    rawCounts = rawCounts.loc[:, ~rawCounts.columns.str.contains(drop_treatment)]
    samplesInfo.loc[samplesInfo['treatment'] == drop_treatment, 'exclude_from_analysis'] = True

#### Drop excluded samples from the data set

In [117]:
samplesInfo = samplesInfo[~samplesInfo['exclude_from_analysis']]
samplesInfo = samplesInfo.drop(columns=['exclude_from_analysis'])

In [118]:
print(samplesInfo['treatment'].unique())
samplesInfo['treatment'].nunique()

['M3' 'EC3' 'EC23' 'EC21' 'EC20' 'EC18' 'EC13' 'Baffled' '1.8%Gluc'
 '1.6%Gluc' '1.5%Suc1%Raf' '1.4%Gluc' '0.5MKCl' '0.5%Raf' '0.2MKCl']


15

In [119]:
print(samplesInfo['assay'].unique())
samplesInfo['assay'].nunique()

['M3-R1' 'M3-R2' 'M3-R3' 'M3-R4' 'EC3-R1' 'EC3-R2' 'EC3-R3' 'EC23-R1'
 'EC23-R2' 'EC23-R3' 'EC21-R1' 'EC21-R2' 'EC21-R3' 'EC20-R1' 'EC20-R2'
 'EC20-R3' 'EC18-R1' 'EC18-R2' 'EC18-R3' 'EC13-R1' 'EC13-R2' 'EC13-R3'
 'Baffled-R1' 'Baffled-R2' '1.8%Gluc-R1' '1.8%Gluc-R2' '1.6%Gluc-R1'
 '1.6%Gluc-R2' '1.5%Suc1%Raf-R1' '1.4%Gluc-R1' '1.4%Gluc-R2' '0.5MKCl-R1'
 '0.5%Raf-R1' '0.5%Raf-R2' '0.2MKCl-R1']


35

------

----------
#### Exclude 0 time points from all assays

In [120]:
t0samples = samplesInfo[samplesInfo['time'] == 0]['sample'].values

In [121]:
samplesInfo = samplesInfo.drop(samplesInfo[samplesInfo['sample'].isin(t0samples)].index)

In [122]:
rawCounts = rawCounts.drop(columns=t0samples)

--------
#### Get barcode sequence along with GC-content for each variant

In [123]:
barcodesInfo = pd.read_csv(rawDataDir+'Kinsler_et_al_2020_BCID_to_barcode_sequence.csv')
barcodesInfo

,barcode,BCID,barcode_fasta,barcode_lowcomplexity,barcode_highcomplexity,set
0,CGCTAAAGACATAATGTGGTTTGTTG_AACGTAAGTCTAAACGTAT...,53,AACGTAAGTCTAAACGTATTTGAGAACGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,AACGTAAGTCTAAACGTATTTGAGAA,Test
1,CGCTAAAGACATAATGTGGTTTGTTG_TCCATAATTGGGAATTGGA...,151,TCCATAATTGGGAATTGGATTTTGGCCGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,TCCATAATTGGGAATTGGATTTTGGC,Train
2,CGCTAAAGACATAATGTGGTTTGTTG_TTAATAAACAAGAAACGGG...,262,TTAATAAACAAGAAACGGGTTGTGGACGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,TTAATAAACAAGAAACGGGTTGTGGA,NaN
3,CGCTAAAGACATAATGTGGTTTGTTG_TCACTAAGAAGGAAGTCCA...,273,TCACTAAGAAGGAAGTCCATTATTACCGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,TCACTAAGAAGGAAGTCCATTATTAC,Test
4,CGCTAAAGACATAATGTGGTTTGTTG_AGCCAAAAAAGTAATATTT...,323,AGCCAAAAAAGTAATATTTTTGACCACGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,AGCCAAAAAAGTAATATTTTTGACCA,NaN
...,...,...,...,...,...,...
526,CGCTAAAGACATAATGTGGTTTGTTG_ACAAGAAGTTGTAAAGGAC...,113483,ACAAGAAGTTGTAAAGGACTTTGGGGCGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,ACAAGAAGTTGTAAAGGACTTTGGGG,NaN
527,CGCTAAAGACATAATGTGGTTTGTTG_ATCTAAAGGTCCAAAAGTA...,134852,ATCTAAAGGTCCAAAAGTATTGTTAGCGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,ATCTAAAGGTCCAAAAGTATTGTTAG,NaN
528,CGCTAAAGACATAATGTGGTTTGTTG_CTCGTAATAAGAAAAATAG...,263665,CTCGTAATAAGAAAAATAGTTGAAGTCGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,CTCGTAATAAGAAAAATAGTTGAAGT,NaN
529,CGCTAAAGACATAATGTGGTTTGTTG_ATAAAAACAAGAATGCCAT...,316954,ATAAAAACAAGAATGCCATTAATATACGCTAAAGACATAATGTGGT...,CGCTAAAGACATAATGTGGTTTGTTG,ATAAAAACAAGAATGCCATTAATATA,NaN


In [124]:
variants_barcodes = []
variants_GCratios = []
for idx, variantInfo in variantsInfo.iterrows():
    variant_barcodeID  = variantInfo['barcode']
    try:
        variant_barcodeSeq     = barcodesInfo.loc[barcodesInfo['BCID'] == variant_barcodeID, 'barcode_highcomplexity'].values[0]
        variant_barcodeGCratio = (variant_barcodeSeq.count('G') + variant_barcodeSeq.count('C'))/len(variant_barcodeSeq)
    except IndexError:
        variant_barcodeSeq     = None
        variant_barcodeGCratio = None
    variants_barcodes.append(variant_barcodeSeq)
    variants_GCratios.append(variant_barcodeGCratio)
    
variantsInfo['barcode_sequence'] = variants_barcodes
variantsInfo['barcode_GCratio']  = variants_GCratios
variantsInfo

,additional_muts,barcode,class,gene,ploidy,type,barcode_sequence,barcode_GCratio
0,TIP1-upstream_point_variant; YKR012C-upstream_...,53,Diploid,Diploid,Diploid,Diploid,AACGTAAGTCTAAACGTATTTGAGAA,0.307692
1,SEH1-missense_variant; ZIP1-missense_variant; ...,151,PKA,IRA1,Haploid,stop_gained,TCCATAATTGGGAATTGGATTTTGGC,0.384615
2,NotSequenced,262,NotSequenced,NotSequenced,NotSequenced,NotSequenced,TTAATAAACAAGAAACGGGTTGTGGA,0.346154
3,NaN,273,PKA,IRA1,Haploid,frameshift_variant,TCACTAAGAAGGAAGTCCATTATTAC,0.346154
4,NotSequenced,323,NotSequenced,NotSequenced,NotSequenced,NotSequenced,AGCCAAAAAAGTAATATTTTTGACCA,0.269231
...,...,...,...,...,...,...,...,...
544,NaN,9000106,PKA,IRA1,Haploid,stop_gained,None,NaN
545,NaN,9000107,PKA,IRA1,Haploid,stop_gained,None,NaN
546,NaN,9000108,PKA,IRA1,Haploid,stop_gained,None,NaN
547,NaN,9000109,PKA,IRA1,Haploid,stop_gained,None,NaN


------

-------
## Save processed data to file

In [125]:
samplesInfo.to_csv(outdir+'samples.csv', index=False)
display(samplesInfo)

,kinsler_sample_label,sample,assay,treatment,rep,time,total_raw_count
1,A1,M3-R1-T1,M3-R1,M3,1,1,4885816.0
2,A2,M3-R1-T2,M3-R1,M3,1,2,8910028.0
3,A3,M3-R1-T3,M3-R1,M3,1,3,1036411.0
4,A4,M3-R1-T4,M3-R1,M3,1,4,5206525.0
6,B1,M3-R2-T1,M3-R2,M3,2,1,1989639.0
...,...,...,...,...,...,...,...
338,BB4,0.5%Raf-R2-T4,0.5%Raf-R2,0.5%Raf,2,4,551741.0
350,O1,0.2MKCl-R1-T1,0.2MKCl-R1,0.2MKCl,1,1,2930331.0
351,O2,0.2MKCl-R1-T2,0.2MKCl-R1,0.2MKCl,1,2,2883775.0
352,O3,0.2MKCl-R1-T3,0.2MKCl-R1,0.2MKCl,1,3,296368.0


In [126]:
variantsInfo.to_csv(outdir+'variants_metadata.csv', index=False)
variantsInfo

,additional_muts,barcode,class,gene,ploidy,type,barcode_sequence,barcode_GCratio
0,TIP1-upstream_point_variant; YKR012C-upstream_...,53,Diploid,Diploid,Diploid,Diploid,AACGTAAGTCTAAACGTATTTGAGAA,0.307692
1,SEH1-missense_variant; ZIP1-missense_variant; ...,151,PKA,IRA1,Haploid,stop_gained,TCCATAATTGGGAATTGGATTTTGGC,0.384615
2,NotSequenced,262,NotSequenced,NotSequenced,NotSequenced,NotSequenced,TTAATAAACAAGAAACGGGTTGTGGA,0.346154
3,NaN,273,PKA,IRA1,Haploid,frameshift_variant,TCACTAAGAAGGAAGTCCATTATTAC,0.346154
4,NotSequenced,323,NotSequenced,NotSequenced,NotSequenced,NotSequenced,AGCCAAAAAAGTAATATTTTTGACCA,0.269231
...,...,...,...,...,...,...,...,...
544,NaN,9000106,PKA,IRA1,Haploid,stop_gained,None,NaN
545,NaN,9000107,PKA,IRA1,Haploid,stop_gained,None,NaN
546,NaN,9000108,PKA,IRA1,Haploid,stop_gained,None,NaN
547,NaN,9000109,PKA,IRA1,Haploid,stop_gained,None,NaN


In [127]:
rawCounts.to_csv(outdir+'counts.csv', index=False)
rawCounts

,M3-R1-T1,M3-R1-T2,M3-R1-T3,M3-R1-T4,M3-R2-T1,M3-R2-T2,M3-R2-T3,M3-R2-T4,M3-R3-T1,M3-R3-T2,M3-R3-T3,M3-R3-T4,M3-R4-T1,M3-R4-T2,M3-R4-T3,M3-R4-T4,EC3-R1-T1,EC3-R1-T2,EC3-R1-T3,EC3-R1-T4,EC3-R2-T1,EC3-R2-T2,EC3-R2-T3,EC3-R2-T4,EC3-R3-T1,EC3-R3-T2,EC3-R3-T3,EC3-R3-T4,EC23-R1-T1,EC23-R1-T2,EC23-R1-T3,EC23-R1-T4,EC23-R2-T1,EC23-R2-T2,EC23-R2-T3,EC23-R2-T4,EC23-R3-T1,EC23-R3-T2,EC23-R3-T3,EC23-R3-T4,EC21-R1-T1,EC21-R1-T2,EC21-R1-T3,EC21-R1-T4,EC21-R2-T1,EC21-R2-T2,EC21-R2-T3,EC21-R2-T4,EC21-R3-T1,EC21-R3-T2,EC21-R3-T3,EC21-R3-T4,EC20-R1-T1,EC20-R1-T2,EC20-R1-T3,EC20-R2-T1,EC20-R2-T2,EC20-R2-T3,EC20-R3-T1,EC20-R3-T2,EC20-R3-T3,EC18-R1-T1,EC18-R1-T2,EC18-R1-T3,EC18-R2-T1,EC18-R2-T2,EC18-R2-T3,EC18-R3-T1,EC18-R3-T2,EC18-R3-T3,EC13-R1-T1,EC13-R1-T2,EC13-R1-T3,EC13-R1-T4,EC13-R2-T1,EC13-R2-T2,EC13-R2-T3,EC13-R2-T4,EC13-R3-T1,EC13-R3-T2,EC13-R3-T3,EC13-R3-T4,Baffled-R1-T1,Baffled-R1-T2,Baffled-R1-T3,Baffled-R1-T4,Baffled-R2-T1,Baffled-R2-T2,Baffled-R2-T3,Baffled-R2-T4,1.8%Gluc-R1-T1,1.8%Gluc-R1-T2,1.8%Gluc-R1-T3,1.8%Gluc-R1-T4,1.8%Gluc-R2-T1,1.8%Gluc-R2-T2,1.8%Gluc-R2-T3,1.8%Gluc-R2-T4,1.6%Gluc-R1-T1,1.6%Gluc-R1-T2,1.6%Gluc-R1-T3,1.6%Gluc-R1-T4,1.6%Gluc-R2-T1,1.6%Gluc-R2-T2,1.6%Gluc-R2-T3,1.6%Gluc-R2-T4,1.5%Suc1%Raf-R1-T1,1.5%Suc1%Raf-R1-T2,1.5%Suc1%Raf-R1-T3,1.5%Suc1%Raf-R1-T4,1.4%Gluc-R1-T1,1.4%Gluc-R1-T2,1.4%Gluc-R1-T3,1.4%Gluc-R1-T4,1.4%Gluc-R2-T1,1.4%Gluc-R2-T2,1.4%Gluc-R2-T3,1.4%Gluc-R2-T4,0.5MKCl-R1-T1,0.5MKCl-R1-T2,0.5MKCl-R1-T3,0.5MKCl-R1-T4,0.5%Raf-R1-T1,0.5%Raf-R1-T2,0.5%Raf-R1-T3,0.5%Raf-R1-T4,0.5%Raf-R2-T1,0.5%Raf-R2-T2,0.5%Raf-R2-T3,0.5%Raf-R2-T4,0.2MKCl-R1-T1,0.2MKCl-R1-T2,0.2MKCl-R1-T3,0.2MKCl-R1-T4
0,6191.0,8361.0,574.0,2148.0,2775.0,17050.0,10057.0,18295.0,8268.0,15562.0,1846.0,11209.0,5916.0,3468.0,3720.0,2945.0,244.0,183.0,81.0,101.0,376.0,164.0,95.0,69.0,223.0,177.0,104.0,79.0,2019.0,2104.0,1529.0,1172.0,2406.0,2892.0,2292.0,1750.0,1856.0,2817.0,1772.0,1686.0,962.0,1883.0,985.0,1025.0,760.0,574.0,1144.0,1070.0,1889.0,1691.0,1945.0,1302.0,2785.0,1855.0,954.0,2493.0,1569.0,1495.0,2199.0,1775.0,1990.0,40.0,365.0,486.0,65.0,102.0,516.0,481.0,574.0,380.0,512.0,594.0,319.0,51.0,700.0,403.0,299.0,70.0,594.0,104.0,361.0,59.0,11213.0,3158.0,908.0,4044.0,8696.0,4901.0,7853.0,4079.0,6157.0,2544.0,380.0,466.0,4670.0,2662.0,363.0,341.0,121.0,113.0,120.0,245.0,102.0,212.0,146.0,116.0,1879.0,442.0,299.0,607.0,5289.0,1933.0,249.0,495.0,4131.0,1553.0,150.0,235.0,11230.0,6963.0,1183.0,1828.0,2713.0,3435.0,198.0,236.0,3076.0,1751.0,167.0,193.0,6361.0,7231.0,780.0,1934.0
1,165765.0,422624.0,62916.0,367191.0,66810.0,915046.0,1357544.0,4230603.0,224264.0,751263.0,187575.0,1956473.0,121243.0,138770.0,293532.0,443175.0,54722.0,102971.0,101137.0,217453.0,88884.0,127795.0,187235.0,209290.0,49070.0,95476.0,150720.0,229493.0,18428.0,40070.0,57945.0,83239.0,21698.0,42476.0,64251.0,93228.0,16341.0,40354.0,43842.0,73539.0,14470.0,50457.0,47319.0,79224.0,11899.0,16668.0,49533.0,76326.0,27178.0,36587.0,58512.0,70336.0,26566.0,35823.0,34463.0,24995.0,31769.0,49171.0,21013.0,25010.0,41128.0,6260.0,179028.0,491772.0,14745.0,49741.0,661420.0,66171.0,171156.0,367527.0,43106.0,160273.0,399304.0,223043.0,54385.0,107883.0,308884.0,241795.0,42068.0,20536.0,267212.0,145296.0,134512.0,75757.0,39864.0,404244.0,123926.0,128460.0,407379.0,497347.0,107559.0,89549.0,20215.0,55134.0,94363.0,105430.0,23514.0,54599.0,5096.0,13055.0,30405.0,141123.0,3330.0,17235.0,32195.0,65247.0,59385.0,17611.0,16887.0,31111.0,136115.0,103853.0,25061.0,105103.0,123119.0,93059.0,15122.0,53362.0,5267.0,237.0,46.0,46.0,98775.0,289984.0,30196.0,67673.0,122581.0,173982.0,31380.0,56512.0,59101.0,47035.0,3860.0,7283.0
2,4856.0,6961.0,523.0,1654.0,2054.0,11295.0,7406.0,12487.0,5251.0,11435.0,1357.0,8488.0,4522.0,2632.0,2995.0,2326.0,1148.0,751.0,280.0,214.0,1873.0,733.0,443.0,205.0,1199.0,703.0,369.0,233.0,1794.0,1776.0,1117.0,935.0,1867.0,2385.0,1902.0,1350.0,1674.0,2276.0,1406.0,1448.0,701.0,1422.0,680.0,897.0,437.0,415.0,761.0,887.0,1650.0,1471.0,1614.0,1146.0,2223.0,1335.0,542.0,2185.0,122